In [76]:
import torch  
import torch.nn as nn  
from torch.autograd import Variable  
import torch.utils.data as Data  
import torchvision  
import time
import numpy as np
#import matplotlib.pyplot as plt  

In [13]:
torch.manual_seed(1)  
  
EPOCH = 1  
BATCH_SIZE = 50  
LR = 0.001  
DOWNLOAD_MNIST = True
if_use_gpu = 1
  
# 获取训练集dataset  
training_data = torchvision.datasets.MNIST(  
             root='./mnist/', # dataset存储路径  
             train=True, # True表示是train训练集，False表示test测试集  
             transform=torchvision.transforms.ToTensor(), # 将原数据规范化到（0,1）区间  
             download=DOWNLOAD_MNIST,  
             )
  
# 打印MNIST数据集的训练集及测试集的尺寸  
print(training_data.train_data.size())  
print(training_data.train_labels.size())  
# torch.Size([60000, 28, 28])  
# torch.Size([60000])  

#plt.imshow(training_data.train_data[0].numpy(), cmap='gray')  
#plt.title('%i' % training_data.train_labels[0])  
#plt.show()  
  
# 通过torchvision.datasets获取的dataset格式可直接可置于DataLoader  
train_loader = Data.DataLoader(dataset=training_data, batch_size=BATCH_SIZE,  
                               shuffle=True)  
  
# 获取测试集dataset  
 
test_data = torchvision.datasets.MNIST(  
             root='./mnist/', # dataset存储路径  
             train=False, # True表示是train训练集，False表示test测试集  
             transform=torchvision.transforms.ToTensor(), # 将原数据规范化到（0,1）区间  
             download=DOWNLOAD_MNIST,  
             )  
# 取前全部10000个测试集样本  
test_x = Variable(torch.unsqueeze(test_data.test_data, dim=1).float(), requires_grad=False)
#test_x = test_x.cuda()
## (~, 28, 28) to (~, 1, 28, 28), in range(0,1)  
test_y = test_data.test_labels
#test_y = test_y.cuda()  

torch.Size([60000, 28, 28])
torch.Size([60000])


In [43]:
# 直接抄来的cnn
class CNN(nn.Module):  
    def __init__(self):  
        super(CNN, self).__init__()  
        self.conv1 = nn.Sequential( # (1,28,28)  
                     nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5,  
                               stride=1, padding=2), # (16,28,28)  
        # 想要con2d卷积出来的图片尺寸没有变化, padding=(kernel_size-1)/2  
                     nn.ReLU(),  
                     nn.MaxPool2d(kernel_size=2) # (16,14,14)  
                     )  
        self.conv2 = nn.Sequential( # (16,14,14)  
                     nn.Conv2d(16, 32, 5, 1, 2), # (32,14,14)  
                     nn.ReLU(),  
                     nn.MaxPool2d(2) # (32,7,7)  
                     )  
        self.out = nn.Linear(32*7*7, 10)  
  
    def forward(self, x):  
        x = self.conv1(x)  
        x = self.conv2(x)  
        x = x.view(x.size(0), -1) # 将（batch，32,7,7）展平为（batch，32*7*7），一个向量
        output = self.out(x)  
        return output  

In [44]:
# 直接抄来的cnn
cnn = CNN()  
if if_use_gpu:
    cnn = cnn.cuda()

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)  
loss_function = nn.CrossEntropyLoss()  
 
 
 
for epoch in range(EPOCH):  
    start = time.time() 
    for step, (x, y) in enumerate(train_loader):  
        b_x = Variable(x, requires_grad=False) 
        b_y = Variable(y, requires_grad=False)  
        if if_use_gpu:
            b_x = b_x.cuda()
            b_y = b_y.cuda()
  
        output = cnn(b_x)  
        loss = loss_function(output, b_y)  
        optimizer.zero_grad()  
        loss.backward()  
        optimizer.step()  
  
        if step % 100 == 0:  
            print('Epoch:', epoch, '|Step:', step,  
                  '|train loss:%.4f'%loss)  
    duration = time.time() - start 
    print('Training duation: %.4f'%duration)
    
cnn = cnn.cpu()
test_output = cnn(test_x)  
pred_y = torch.max(test_output, 1)[1].data.squeeze()
accuracy = sum(pred_y.numpy()==test_y.numpy())/len(test_y.numpy())

Epoch: 0 |Step: 0 |train loss:2.3099
Epoch: 0 |Step: 100 |train loss:0.5186
Epoch: 0 |Step: 200 |train loss:0.1705
Epoch: 0 |Step: 300 |train loss:0.0541
Epoch: 0 |Step: 400 |train loss:0.0901
Epoch: 0 |Step: 500 |train loss:0.0945
Epoch: 0 |Step: 600 |train loss:0.1182
Epoch: 0 |Step: 700 |train loss:0.0937
Epoch: 0 |Step: 800 |train loss:0.1940
Epoch: 0 |Step: 900 |train loss:0.0278
Epoch: 0 |Step: 1000 |train loss:0.0342
Epoch: 0 |Step: 1100 |train loss:0.1762
Training duation: 9.2809


In [28]:
# 自己修改成LeNet-CNN
class CNN(nn.Module):  
    def __init__(self):  
        super(CNN, self).__init__()  
        self.conv1 = nn.Sequential( # (1,28,28)  
                      nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2), # (6,28,28)  
                      nn.ReLU(),  
                      nn.MaxPool2d(kernel_size=2) # (6,14,14)  
                      )  
        self.conv2 = nn.Sequential( # (6,14,14)  
                      nn.Conv2d(6, 16, 5, 1, 0), # (16,10,10)  
                      nn.ReLU(),  
                      nn.MaxPool2d(2) # (16,5,5)  
                     )
        self.conv3 = nn.Sequential( # (16,5,5)   
                      nn.Conv2d(16, 120, 5, 1, 0), # (120,1,1)  
                      nn.ReLU()
                     )
        
        self.fc   = nn.Sequential( # (120,1,1)
                      nn.Linear(120*1*1, 84),  # (84,1,1)
                      nn.ReLU()
                     )
        self.out   = nn.Linear(84*1*1, 10)
  
    def forward(self, x):  
        x = self.conv1(x)  
        x = self.conv2(x) 
        x = self.conv3(x)
        x = x.view(x.size(0), -1) # 将（batch，32,7,7）展平为（batch，32*7*7），一个向量
        x = self.fc(x)
        output = self.out(x)  
        return output  

In [30]:
# 自己修改成LeNet-CNN
cnn = CNN()  
if if_use_gpu:
    cnn = cnn.cuda()

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)  
loss_function = nn.CrossEntropyLoss()  

for epoch in range(EPOCH):  
    start = time.time() 
    for step, (x, y) in enumerate(train_loader):  
        b_x = Variable(x, requires_grad=False) 
        b_y = Variable(y, requires_grad=False)  
        if if_use_gpu:
            b_x = b_x.cuda()
            b_y = b_y.cuda()
  
        output = cnn(b_x)  
        loss = loss_function(output, b_y)  
        optimizer.zero_grad()  
        loss.backward()  
        optimizer.step()  
  
        if step % 100 == 0:  
            print('Epoch:', epoch, '|Step:', step,  
                  '|train loss:%.4f'%loss)
    duration = time.time() - start 
    print('Training duation: %.4f'%duration)
    
cnn = cnn.cpu()
test_output = cnn(test_x)  
pred_y = torch.max(test_output, 1)[1].data.squeeze() 
# torch.max(D, 0or1) axis=0，返回D中每列的最大值及其行索引；axis=1，返回D中每行的最大值及其列索引
accuracy = sum(pred_y.numpy()==test_y.numpy())/len(test_y.numpy())

Epoch: 0 |Step: 0 |train loss:2.3225
Epoch: 0 |Step: 100 |train loss:0.4402
Epoch: 0 |Step: 200 |train loss:0.2708
Epoch: 0 |Step: 300 |train loss:0.1160
Epoch: 0 |Step: 400 |train loss:0.2075
Epoch: 0 |Step: 500 |train loss:0.1879
Epoch: 0 |Step: 600 |train loss:0.1099
Epoch: 0 |Step: 700 |train loss:0.1298
Epoch: 0 |Step: 800 |train loss:0.0946
Epoch: 0 |Step: 900 |train loss:0.1945
Epoch: 0 |Step: 1000 |train loss:0.1123
Epoch: 0 |Step: 1100 |train loss:0.3685
Training duation: 11.3302


In [56]:
import os
from skimage import io
import torchvision.datasets.mnist as mnist

root="C:/Users/jxjsj/Desktop/JupyterHome/Data/fashion/"
train_set = (
    mnist.read_image_file(os.path.join(root, 'train-images-idx3-ubyte')),
    mnist.read_label_file(os.path.join(root, 'train-labels-idx1-ubyte'))
        )
test_set = (
    mnist.read_image_file(os.path.join(root, 't10k-images-idx3-ubyte')),
    mnist.read_label_file(os.path.join(root, 't10k-labels-idx1-ubyte'))
        )
print("training set :",train_set[0].size())
print("test set :",test_set[0].size())

def convert_to_img(train=True):
    if(train):
        f=open(root+'train.txt','w')
        data_path=root+'/train/'
        if(not os.path.exists(data_path)):
            os.makedirs(data_path)
        for i, (img,label) in enumerate(zip(train_set[0],train_set[1])):
            img_path=data_path+str(i)+'.jpg'
            io.imsave(img_path,img.numpy())
            f.write(img_path+' '+str(label)+'\n')
        f.close()
    else:
        f = open(root + 'test.txt', 'w')
        data_path = root + '/test/'
        if (not os.path.exists(data_path)):
            os.makedirs(data_path)
        for i, (img,label) in enumerate(zip(test_set[0],test_set[1])):
            img_path = data_path+ str(i) + '.jpg'
            io.imsave(img_path, img.numpy())
            f.write(img_path + ' ' + str(label) + '\n')
        f.close()

convert_to_img(True)
convert_to_img(False)

# 这样就会在e:/fashion_mnist/目录下分别生成train和test文件夹，用于存放图片。还在该目录下生成了标签文件train.txt和test.txt.

training set : torch.Size([60000, 28, 28])
test set : torch.Size([10000, 28, 28])


In [93]:
# 二、进行CNN分类训练和测试

from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image


def default_loader(path):
    return Image.open(path).convert('RGB')


class MyDataset(Dataset):
    def __init__(self, txt, transform=None, target_transform=None, loader=default_loader):
        fh = open(txt, 'r')
        imgs = []
        for line in fh:
            line = line.strip('\n')
            line = line.rstrip()
            words = line.split()
            imgs.append((words[0],int(words[1][7]))) # 看情况修改
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = self.loader(fn)
        if self.transform is not None:
            img = self.transform(img)
        return img,label

    def __len__(self):
        return len(self.imgs)

In [114]:
root = 'C:/Users/jxjsj/Desktop/JupyterHome/Data/fashion/'
train_data = MyDataset(txt=root+'train.txt', transform=transforms.ToTensor())
train_data_loader = DataLoader(train_data, batch_size=200,shuffle=True)
test_data = MyDataset(txt=root+'test.txt', transform=transforms.ToTensor())
test_data_loader = DataLoader(test_data, batch_size=test_data.__len__(),shuffle=True)

In [115]:
#-----------------create the Net and training------------------------

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, 3, 1, 1), #32 28 28
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)) #32 14 14
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, 3, 1, 1), #64 14 14
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2) #64 7 7
        )
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 64, 3, 1, 1), #64 7 7
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2) #64 3 3 除法是floor除，取整数部分
        )
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(64 * 3 * 3, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 10)
        )

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv2_out = self.conv2(conv1_out)
        conv3_out = self.conv3(conv2_out)
        res = conv3_out.view(conv3_out.size(0), -1)
        out = self.dense(res)
        return out

In [125]:
model = Net()

optimizer = torch.optim.Adam(model.parameters())
loss_func = torch.nn.CrossEntropyLoss()

for epoch in range(2):
    print('epoch {}'.format(epoch + 1))
    # training-----------------------------
    train_acc = 0.
    for step, (batch_x, batch_y) in enumerate(train_data_loader):
        batch_x, batch_y = Variable(batch_x), Variable(batch_y)
        out = model(batch_x)
        loss = loss_func(out, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pred = torch.max(out, 1)[1]
        num_correct = (pred == batch_y).sum()
        train_acc += num_correct.data.item()
        print('Step:',step+1,'Finished!')
    print('Acc: {:.6f}'.format(train_acc / (len(train_data))))

    # evaluation--------------------------------
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for batch_x, batch_y in test_data_loader:
        batch_x, batch_y = Variable(batch_x, volatile=True), Variable(batch_y, volatile=True)
        out = model(batch_x)
        loss = loss_func(out, batch_y)
        pred = torch.max(out, 1)[1]
        num_correct = (pred == batch_y).sum()
        eval_acc += num_correct.data.item()
    print('Acc: {:.6f}'.format(eval_acc / (len(test_data))))

epoch 1
Step: 0 Finished!
Step: 1 Finished!
Step: 2 Finished!
Step: 3 Finished!
Step: 4 Finished!
Step: 5 Finished!
Step: 6 Finished!
Step: 7 Finished!
Step: 8 Finished!
Step: 9 Finished!
Step: 10 Finished!
Step: 11 Finished!
Step: 12 Finished!
Step: 13 Finished!
Step: 14 Finished!
Step: 15 Finished!
Step: 16 Finished!
Step: 17 Finished!
Step: 18 Finished!
Step: 19 Finished!
Step: 20 Finished!
Step: 21 Finished!
Step: 22 Finished!
Step: 23 Finished!
Step: 24 Finished!
Step: 25 Finished!
Step: 26 Finished!
Step: 27 Finished!
Step: 28 Finished!
Step: 29 Finished!
Step: 30 Finished!
Step: 31 Finished!
Step: 32 Finished!
Step: 33 Finished!
Step: 34 Finished!
Step: 35 Finished!
Step: 36 Finished!
Step: 37 Finished!
Step: 38 Finished!
Step: 39 Finished!
Step: 40 Finished!
Step: 41 Finished!
Step: 42 Finished!
Step: 43 Finished!
Step: 44 Finished!
Step: 45 Finished!
Step: 46 Finished!
Step: 47 Finished!
Step: 48 Finished!
Step: 49 Finished!
Step: 50 Finished!
Step: 51 Finished!
Step: 52 Finis

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


RuntimeError: $ Torch: not enough memory: you tried to allocate 0GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201